# HW2: Problem 2: Working out Backpropagation

Read Chapter 2 of Michael Nielsen's article/book from top to bottom:

* [http://neuralnetworksanddeeplearning.com/chap2.html](http://neuralnetworksanddeeplearning.com/chap2.html)

He outlines a few exersizes in that article which you must complete. Do the following a, b, c:

a. He invites you to write out a proof of equation BP3

b. He invites you to write out a proof of equation BP4

c. He proposes that you code a fully matrix-based approach to backpropagation over a mini-batch. Implement this with explanation where you change the notation so that instead of having a bias term, you assume that the input variables are augmented with a "column" of "1"s, and that the weights $w_0$.

Your submission should be a single jupyter notebook. Use markdown cells with latex for equations of a jupyter notebook for each proof for "a." and "b.". Make sure you include text that explains your steps. Next for "c" this is an implementation problem. You need to understand and modify the code the Michael Nielsen provided so that instead it is using a matrixed based approach. Again don't keep the biases separate. After reading data in (use the iris data set), create a new column corresponding to $x_0=1$, and as mentioned above and discussed in class (see notes) is that the bias term can then be considered a weight $w_0$. Again use markdown cells around your code and comments to explain your work. Test the code on the iris data set with 4 node input (5 with a constant 1), three hidden nodes, and three output nodes, one for each species/class.

## a. Proof of Michael Nielsons equation BP3

BP3:
$$
  \frac{∂C}{∂b^l_{j}}=δ^l_{j}
$$

Recall by definition that:
$$
  δ^l_{j}=\frac{∂C}{∂z^l_{j}}
$$
<br>
$$
  z^l_{j}≡Σ_{k} w^l_{jk}a^{l−1}_{k}+b^l_{j}
$$
<br>
Taking the derivative of the intermediate quantity,𝒵 respect to bias and we get:
<br>
$$
  \frac{∂z^l_{j}}{∂b^l_{j}}=1
$$
<br>
Multiplying this to both side of the error equation gives the following:
<br>
$$
  δ^l_{j}=\frac{∂C}{∂z^l_{j}}\frac{∂z^l_{j}}{∂b^l_{j}}
$$
<br>
Simplify and we get the following, which is the equation for BP3:
<br>
$$
  δ^l_{j}=\frac{∂C}{∂b^l_{j}}
$$
<br>

## b. Proof of Michael Nielsons equation BP4

BP3:
$$
  \frac{∂C}{∂w^l_{jk}}=a^{l-1}_kδ^l_{j}
$$

Recall by definition that:
$$
  δ^l_{j}=\frac{∂C}{∂z^l_{j}}
$$
<br>
$$
  z^l_{j}≡Σ_{k} w^l_{jk}a^{l−1}_{k}+b^l_{j}
$$
<br>
Taking the derivative of the intermediate quantity,𝒵 respect to the weight and we get:
<br>
$$
  \frac{∂z^l_{j}}{∂w^l_{jk}}=a^{l-1}_k
$$
<br>
Multiplying this to both side of the error equation gives the following:
<br>
$$
  δ^l_{j}a^{l-1}_k=\frac{∂C}{∂z^l_{j}}\frac{∂z^l_{j}}{∂w^l_{jk}}
$$
<br>
Simplify and we get the following, which is the equation for BP3:
<br>
$$
  δ^l_{j}a^{l-1}_k=\frac{∂C}{∂w^l_{jk}}
$$
<br>

## c. Using both markdown cells and code cells implement that you code a fully matrix-based approach to backpropagation over a mini-batch. Implement this with explanation where you change the notation so that instead of having a bias term, you assume that the input variables are augmented with a "column" of "1"s, and that the weights $w_0$.

Modify the shape of **mini_batch** and **delta** to accomplish the computation with inputs in a matrix form.
Also, added codes to reshape the input and other values during calculuation to match the dimensions

In [ ]:
import numpy as np
import random

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a.reshape(a.shape[0], 1))+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data[0])
        for j in range(epochs):
            mini_batches = []
            for i in range(n // mini_batch_size):
                temp = (training_data[0][i*mini_batch_size:(i+1)*mini_batch_size], training_data[1][i*mini_batch_size:(i+1)*mini_batch_size])
                mini_batches.append(temp)
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        x, y = mini_batch
        new_x = np.array(x).reshape(4, len(x))
        new_y = np.array(y).reshape(len(y), 1)
        delta_nabla_b, delta_nabla_w = self.backprop(new_x, new_y)
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        # Changed the shape of delta in order to perform matrix based.
        # delta is no longer a scalar value
        # delta now is a matrix
        delta = np.array([self.cost_derivative(act[-1], y).reshape(1, len(y)) * sig.reshape(1, len(sig)) \
             for act, sig in zip(activations[-1], sigmoid_prime(zs[-1]))])
        delta = delta.reshape(-1,delta.shape[-1])

        nabla_b[-1] = delta.sum(1).reshape([len(delta), 1])
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta.sum(1).reshape([len(delta), 1])
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""

        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

In [ ]:
from sklearn.model_selection import train_test_split
X = np.array(iris['data'])
y = np.array(iris['target'])
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], test_size=0.20, random_state=42)
training_data = [X_train, y_train]
testing_data = [(x, y) for x, y in zip(X_test, y_test)]

network = Network([4, 3, 3, 3])
network.SGD(training_data, 500, 30, .1, testing_data)

Epoch 0: 9 / 30
Epoch 1: 9 / 30
Epoch 2: 9 / 30
Epoch 3: 9 / 30
Epoch 4: 9 / 30
Epoch 5: 9 / 30
Epoch 6: 9 / 30
Epoch 7: 9 / 30
Epoch 8: 9 / 30
Epoch 9: 9 / 30
Epoch 10: 9 / 30
Epoch 11: 9 / 30
Epoch 12: 9 / 30
Epoch 13: 9 / 30
Epoch 14: 9 / 30
Epoch 15: 9 / 30
Epoch 16: 9 / 30
Epoch 17: 9 / 30
Epoch 18: 5 / 30
Epoch 19: 4 / 30
Epoch 20: 4 / 30
Epoch 21: 2 / 30
Epoch 22: 2 / 30
Epoch 23: 2 / 30
Epoch 24: 2 / 30
Epoch 25: 2 / 30
Epoch 26: 2 / 30
Epoch 27: 2 / 30
Epoch 28: 2 / 30
Epoch 29: 2 / 30
Epoch 30: 1 / 30
Epoch 31: 1 / 30
Epoch 32: 1 / 30
Epoch 33: 1 / 30
Epoch 34: 1 / 30
Epoch 35: 1 / 30
Epoch 36: 1 / 30
Epoch 37: 1 / 30
Epoch 38: 1 / 30
Epoch 39: 1 / 30
Epoch 40: 1 / 30
Epoch 41: 1 / 30
Epoch 42: 1 / 30
Epoch 43: 1 / 30
Epoch 44: 1 / 30
Epoch 45: 1 / 30
Epoch 46: 1 / 30
Epoch 47: 1 / 30
Epoch 48: 1 / 30
Epoch 49: 1 / 30
Epoch 50: 1 / 30
Epoch 51: 1 / 30
Epoch 52: 1 / 30
Epoch 53: 1 / 30
Epoch 54: 1 / 30
Epoch 55: 1 / 30
Epoch 56: 1 / 30
Epoch 57: 1 / 30
Epoch 58: 1 / 30
Epoch 5